In [1]:
import pandas as pd

In [2]:
# 1, 获取数据
data = pd.read_csv("./FBlocation/train.csv")


FileNotFoundError: [Errno 2] No such file or directory: './FBlocation/train.csv'

In [3]:
# 2. 基本的数据处理
# 1) 缩小数据范围
data.query("x<2.5 & x>2 & y<1.5 & y>1.0")

,row_id,x,y,accuracy,time,place_id
112,112,2.2360,1.3655,66,623174,7663031065
180,180,2.2003,1.2541,65,610195,2358558474
367,367,2.4108,1.3213,74,579667,6644108708
874,874,2.0822,1.1973,320,143566,3229876087
1022,1022,2.0160,1.1659,65,207993,3244363975
...,...,...,...,...,...,...
29115112,29115112,2.1889,1.2914,168,721885,4606837364
29115204,29115204,2.1193,1.4692,58,563389,2074133146
29115338,29115338,2.0007,1.4852,25,765986,6691588909
29115464,29115464,2.4132,1.4237,61,151918,7396159924


In [4]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [5]:
# 2) 处理时间特征
time_value = pd.to_datetime(data["time"], unit="s")


In [6]:
date = pd.DatetimeIndex(time_value)


In [7]:
data["day"] = date.day


In [8]:
data["weekday"] = date.weekday


In [9]:
data["hour"] = date.hour


In [10]:
data.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
0,0,0.7941,9.0809,54,470702,8523065625,6,1,10
1,1,5.9567,4.7968,13,186555,1757726713,3,5,3
2,2,8.3078,7.0407,74,322648,1137537235,4,6,17
3,3,7.3665,2.5165,65,704587,6567393236,9,4,3
4,4,4.0961,1.1307,31,472130,7440663949,6,1,11


In [11]:
# 3) 过滤签到次数少的地点
place_count = data.groupby("place_id").count()["row_id"]


In [12]:
place_count

place_id
1000015801     78
1000017288     95
1000025138    563
1000052096    961
1000063498     60
             ... 
9999851158     60
9999855083    212
9999862567     63
9999916757    508
9999932225    218
Name: row_id, Length: 108390, dtype: int64

In [13]:
place_count[place_count > 3].head()


place_id
1000015801     78
1000017288     95
1000025138    563
1000052096    961
1000063498     60
Name: row_id, dtype: int64

In [15]:
data_final.head()

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
0,0,0.7941,9.0809,54,470702,8523065625,6,1,10
1,1,5.9567,4.7968,13,186555,1757726713,3,5,3
2,2,8.3078,7.0407,74,322648,1137537235,4,6,17
3,3,7.3665,2.5165,65,704587,6567393236,9,4,3
4,4,4.0961,1.1307,31,472130,7440663949,6,1,11


In [16]:
# 筛选特征值和目标值
x = data_final[["x", "y", "accuracy", "day", "weekday", "hour"]]
y = data_final["place_id"]


In [17]:
x.head()

,x,y,accuracy,day,weekday,hour
0,0.7941,9.0809,54,6,1,10
1,5.9567,4.7968,13,3,5,3
2,8.3078,7.0407,74,4,6,17
3,7.3665,2.5165,65,9,4,3
4,4.0961,1.1307,31,6,1,11


In [18]:
y.head()

0    8523065625
1    1757726713
2    1137537235
3    6567393236
4    7440663949
Name: place_id, dtype: int64

In [19]:
# 数据集划分
from sklearn.model_selection import train_test_split

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y)


In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [23]:
# 3) 特征工程: 标准化
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

# 4) KNN 算法预估器
estimator = KNeighborsClassifier()

# 加入网络搜索与交叉验证
# 参数准备
param_dict = {"n_neighbors": [3, 5, 7, 9]}
estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)
estimator.fit(x_train, y_train)

# 5) 模型评估
# 方法1: 直接对比真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict: \n", y_predict)
print("直接对比真实值和预测值: \n", y_test == y_predict)

# 方法2: 计算准确率
score = estimator.score(x_test, y_test)
print("准确率为: \n", score)

print("最佳参数为: \n", estimator.best_params_)
print("最佳结果为: \n", estimator.best_score_)
print("最佳估计器为: \n", estimator.best_estimator_)
print("最终交叉验证结果为: \n", estimator.cv_results_)


/opt/homebrew/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


KeyboardInterrupt: 